**Objective : Predict whether the cancer is benign or malignant**

## Part-1: Data Pre-Processing

> Dataset link: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

#### Importing requisite libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Ignore warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### Importing dataset

In [ ]:
dataset = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

#### Data Exploration

In [ ]:
dataset.shape

There are 33 columns and 569 observations in the dataframe

In [ ]:
dataset.info()

- We can observe that 31 columns are numeric, and the target variable `diagnosis` is an object.
- The column `Unnamed: 32`, has no observations
- All other columns do not have null values

##### Analyzing categorical variables

In [ ]:
dataset.select_dtypes(include = "object").columns

In [ ]:
# Checking number of categorical variables
len(dataset.select_dtypes(include = "object").columns)

There is only one column `diagnosis` which is a categorical variable

##### Checking numerical variables

In [ ]:
dataset.select_dtypes(include=["int64","float64"]).columns

In [ ]:
# Checking number of numerical variables
len(dataset.select_dtypes(include=["int64","float64"]).columns)

There are 32 numerical variables

##### Get statistical summary for numerical variables

In [ ]:
dataset.describe()

In [ ]:
# Get the list of columns
dataset.columns

### Deal with missing values

Check if we have any null values in the dataset

In [ ]:
dataset.isnull().values.any()

Get Total number of null values in the dataset

In [ ]:
dataset.isnull().values.sum()

Getting the column(s) which have null values

In [ ]:
dataset.columns[dataset.isnull().any()]

In [ ]:
len(dataset.columns[dataset.isnull().any()])

Checking total null values in the column `'Unnamed: 32'`

In [ ]:
dataset['Unnamed: 32'].count()

We are dropping the column 'Unnamed: 32' as it only contains NULL values

In [ ]:
dataset = dataset.drop(columns = 'Unnamed: 32')

Check the shape of dataset after dropping column

In [ ]:
dataset.shape

Checking to see if there are any NULL values in the dataset

In [ ]:
dataset.isnull().values.any()

### Dealing with Categorical Data

In [ ]:
dataset.select_dtypes(include = "object").columns

Getting unique values in `diagnosis` column

In [ ]:
dataset["diagnosis"].unique()

In [ ]:
# Total number of unique values
dataset["diagnosis"].nunique()

##### Perform one-hot encoding to convert `diagnosis` to a numerical variable

In [ ]:
dataset = pd.get_dummies(data = dataset, drop_first = True)

In [ ]:
dataset.head()

### Create a CountPlot to check `diagnosis_M`

In [ ]:
sns.countplot(data = dataset, x = "diagnosis_M")
plt.show()

In [ ]:
# Benign (0) values
(dataset["diagnosis_M"] == 0).sum()

In [ ]:
# Benign (0) values
(dataset["diagnosis_M"] == 1).sum()

### Correlation Matrix and HeatMap

Dropping Target variable `diagnosis_M`

In [ ]:
dataset_2 = dataset.drop(columns = "diagnosis_M")

In [ ]:
dataset_2.head()

Creating correlation b/w Target Variable `diagnosis_M` and other independent variables

In [ ]:
dataset_2.corrwith(dataset['diagnosis_M']).plot.bar(
figsize = (20,10), title = "Correlation with diagnosis_M", rot = 45, grid = True)
plt.show()

Creating a correlation matrix `corr` to view the results better

In [ ]:
corr = dataset_2.corr()

In [ ]:
corr

Creating a HeatMap to view the correlations

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(data = corr, annot = True, cmap = "RdYlGn")
plt.show()

### Splitting the dataset into train and test set

In [ ]:
dataset.head()

#### Matrix of Features / Independent Variables

Dropping `id` and `diagnosis_M` fields from the dataset

In [ ]:
x = dataset.iloc[:, 1:-1].values

In [ ]:
x.shape

#### Target Variable / Dependent Variable

In [ ]:
y = dataset.iloc[:,-1].values

In [ ]:
y.shape

#### Split data using scikit-learn `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train

In [ ]:
x_test

## Part-2: Building the model

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier_lr = LogisticRegression(random_state=0)

In [ ]:
classifier_lr.fit(x_train,y_train)

In [ ]:
y_pred = classifier_lr.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

In [ ]:
results = pd.DataFrame([['Logistic Regression', acc, f1, prec, rec]],
               columns = ['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])

In [ ]:
results

Creating a Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
accuracies = cross_val_score(estimator=classifier_lr, X=x_train, y=y_train, cv=10)

In [ ]:
print("Accuracy is {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation is {:.2f} %".format(accuracies.std()*100))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier_rm = RandomForestClassifier(random_state=0)
classifier_rm.fit(x_train, y_train)

In [ ]:
y_pred = classifier_rm.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

In [ ]:
model_results = pd.DataFrame([['Random forest', acc, f1, prec, rec]],
               columns = ['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])

In [ ]:
results = results.append(model_results, ignore_index=True)

In [ ]:
results

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator=classifier_rm, X=x_train, y=y_train, cv=10)

print("Accuracy is {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation is {:.2f} %".format(accuracies.std()*100))

## Part-3: Randomized Search to find the best Parameters

**We will be using `Logistic Regression` as that seems to be the best model for our scenario**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
parameters = {'penalty':['l1', 'l2', 'elasticnet', 'none'],
              'C':[0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
              }

In [ ]:
parameters

In [ ]:
random_search = RandomizedSearchCV(estimator=classifier_lr,param_distributions=parameters, n_iter=5, 
                                   scoring='roc_auc', n_jobs = -1, cv=5, verbose=3)

In [ ]:
random_search.fit(x_train, y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_score_

In [ ]:
random_search.best_params_

## Part-4: Final Model (Logistic Regression)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=0.75, random_state=0, solver='saga',penalty='l2')

In [ ]:
classifier.fit(x_train, y_train)

In [ ]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)

model_results = pd.DataFrame([['Final Logistic Regression', acc, f1, prec, rec]],
               columns = ['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])


results = results.append(model_results, ignore_index = True)
results

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator=classifier, X=x_train, y=y_train, cv=10)

print("Accuracy is {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation is {:.2f} %".format(accuracies.std()*100))

### Part-5: Predicting a single observation

In [ ]:
dataset.head()

In [ ]:
out_list = dataset.iloc[0,:1:-1].to_list()

In [ ]:
np_single_observation = np.array(out_list,ndmin=2)

In [ ]:
np_single_observation

In [ ]:
classifier.predict(sc.transform(np_single_observation))

***For this sample observation, the model is predicting that the cancer is `malignant`***

In [ ]:
# Checking actual data
dataset.iloc[0,[1,-1]].to_list()

***For this sample observation, the actual data is also predicting that the cancer is `malignant`***